In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D,Dropout,Flatten,Dense
from tensorflow.keras.models import Sequential

In [2]:

def ConsineSim(img1,img2):
    a = np.matmul(np.transpose(img1,img2))
    b = np.sum(np.multiply(img1,img1))
    c = np.sum(np.multiply(img2,img2))
    return 1-(a/(np.sqrt(b)*np.sqrt(c)))

In [3]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224,3))),
model.add(Convolution2D(64,(3,3),activation="relu")),
model.add(ZeroPadding2D((1,1))),
model.add(Convolution2D(64,(3,3),activation="relu")),
model.add(MaxPooling2D((2,2), strides=(2,2))),

model.add(ZeroPadding2D((1,1))),
model.add(Convolution2D(128,(3,3),activation="relu")),
model.add(ZeroPadding2D((1,1))),
model.add(Convolution2D(128,(3,3),activation="relu")),
model.add(MaxPooling2D((2,2), strides=(2,2))),

model.add(ZeroPadding2D((1,1))),
model.add(Convolution2D(256,(3,3),activation="relu")),
model.add(ZeroPadding2D((1,1))),
model.add(Convolution2D(256,(3,3),activation="relu")),
model.add(ZeroPadding2D((1,1))),
model.add(Convolution2D(256,(3,3),activation="relu")),
model.add(MaxPooling2D((2,2), strides=(2,2))),

model.add(ZeroPadding2D((1,1))),
model.add(Convolution2D(512,(3,3),activation="relu")),
model.add(ZeroPadding2D((1,1))),
model.add(Convolution2D(512,(3,3),activation="relu")),
model.add(ZeroPadding2D((1,1))),
model.add(Convolution2D(512,(3,3),activation="relu")),
model.add(MaxPooling2D((2,2), strides=(2,2))),

model.add(Convolution2D(4096,(7,7),activation="relu")),
model.add(Dropout(0.5)),
model.add(Convolution2D(4096,(1,1),activation="relu")),
model.add(Dropout(0.5)),
model.add(Convolution2D(2048,(1,1))),
model.add(Flatten()),
model.add(Dense(1,activation="softmax")),
vectorizer= tf.keras.Model(inputs=model.layers[0].input,outputs=model.layers[-2].output)
# this will give the feature vector when used for prediction for both cropped input images
# we will use cosine similarity to detect similarity in faces.

In [4]:
vectorizer

In [5]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [6]:
def load_image(x):
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [7]:
pos_data = tf.data.Dataset.list_files('cropped_faces\\face3\\*.jpg', shuffle=False)
pos_data = pos_data.map(load_image)
pos_data = pos_data.map(lambda x: x/255)
pos_data = pos_data.map(lambda x: tf.image.resize(x, (224,224)))
# adding label 1 to positive train data
pos_labels= pos_data.map(lambda x:np.array([1]))	

pos_data=tf.data.Dataset.zip(pos_data,pos_labels)
pos_data=pos_data.batch(8)

neg_data = tf.data.Dataset.list_files('cropped_faces\\face2\\*.jpg', shuffle=False)
neg_data = neg_data.map(load_image)
neg_data = neg_data.map(lambda x: x/255)
neg_data = neg_data.map(lambda x: tf.image.resize(x, (224,224)))
# adding label 0 to negative train data
neg_labels= neg_data.map(lambda x:np.array([0]))	

neg_data=tf.data.Dataset.zip(neg_data,neg_labels)
neg_data=neg_data.batch(8)

dataset = pos_data.concatenate(neg_data)
dataset.shuffle(3, reshuffle_each_iteration=True)


val_data = tf.data.Dataset.list_files('cropped_faces\\face1\\*.jpg', shuffle=False)
val_data = val_data.map(load_image)
val_data = val_data.map(lambda x: x/255)
val_data = val_data.map(lambda x: tf.image.resize(x, (224,224)))
val_labels = val_data.map(lambda x: np.array([1]))
val_data = tf.data.Dataset.zip(val_data,val_labels)
val_data=val_data.batch(8)

In [8]:
dataset

<_ConcatenateDataset element_spec=(TensorSpec(shape=(None, 224, 224, None), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))>

In [9]:
model.fit(dataset,validation_split=0.2)

10/10 [==============================] - 98s 9s/step - loss: 11456.7812 - accuracy: 0.5070


In [15]:
features=dataset.as_numpy_iterator().next()
features

(array([[[[0.9278737 , 0.95371026, 0.9372549 ],
          [0.92156863, 0.95686275, 0.9372549 ],
          [0.9264644 , 0.9499938 , 0.9343075 ],
          ...,
          [0.7336897 , 0.74545443, 0.7101603 ],
          [0.73353964, 0.74530435, 0.71785337],
          [0.73333335, 0.74509805, 0.7176471 ]],
 
         [[0.92941177, 0.9529412 , 0.94509804],
          [0.92156863, 0.95686275, 0.94509804],
          [0.9254902 , 0.9490196 , 0.9411765 ],
          ...,
          [0.7320028 , 0.7437675 , 0.7084734 ],
          [0.73333335, 0.74509805, 0.7176471 ],
          [0.73333335, 0.74509805, 0.7176471 ]],
 
         [[0.92682076, 0.95035017, 0.9346639 ],
          [0.92682076, 0.95035017, 0.9346639 ],
          [0.9189776 , 0.95427173, 0.9346639 ],
          ...,
          [0.73504525, 0.74680996, 0.71151584],
          [0.73333335, 0.74509805, 0.7176471 ],
          [0.7297682 , 0.7415329 , 0.71408194]],
 
         ...,
 
         [[0.23788512, 0.24180669, 0.31239495],
          [0.22352

In [12]:
feature_vectors=vectorizer(features)

ValueError: Layer "model" expects 1 input(s), but it received 2 input tensors. Inputs received: [<tf.Tensor: shape=(8, 224, 224, 3), dtype=float32, numpy=
array([[[[0.9278737 , 0.95371026, 0.9372549 ],
         [0.92156863, 0.95686275, 0.9372549 ],
         [0.9264644 , 0.9499938 , 0.9343075 ],
         ...,
         [0.7336897 , 0.74545443, 0.7101603 ],
         [0.73353964, 0.74530435, 0.71785337],
         [0.73333335, 0.74509805, 0.7176471 ]],

        [[0.92941177, 0.9529412 , 0.94509804],
         [0.92156863, 0.95686275, 0.94509804],
         [0.9254902 , 0.9490196 , 0.9411765 ],
         ...,
         [0.7320028 , 0.7437675 , 0.7084734 ],
         [0.73333335, 0.74509805, 0.7176471 ],
         [0.73333335, 0.74509805, 0.7176471 ]],

        [[0.92682076, 0.95035017, 0.9346639 ],
         [0.92682076, 0.95035017, 0.9346639 ],
         [0.9189776 , 0.95427173, 0.9346639 ],
         ...,
         [0.73504525, 0.74680996, 0.71151584],
         [0.73333335, 0.74509805, 0.7176471 ],


In [19]:
def verify(img1,img2):
    vec1 = model.predict(img1)
    vec2 = model.predict(img2)
    sim = ConsineSim(vec1,vec2)
    threshold = 0.4
    
    f = plt.figure()
    f.add_subplot(1,2,1)
    plt.imshow(vec1)
    plt.xticks([])
    plt.yticks([])
    f.add_subplot(1,2,2)
    plt.imshow(vec2)
    plt.xticks([])
    plt.yticks([])
    plt.show(block=True)
    
    print("Cosine Similarity: ",sim) 
    if sim<threshold:
        print("They are same person")
    else:
        print("They are not same person") 

In [22]:
img1 = cv2.imread("./IMG_20230427_111749_copy.jpg")
img1 = cv2.resize(img1,(224,224))
img1 = tf.expand_dims(img1,0)

img2 = cv2.imread("./IMG_20230427_111749_copy.jpg")
img2 = cv2.resize(img2,(224,224))
img2 = tf.expand_dims(img2,0)

In [25]:
img1

TensorShape([1, 224, 224, 3])

In [24]:
ConsineSim(img1,img2)

TypeError: only integer scalar arrays can be converted to a scalar index